In [1]:
# pdf plumber
import requests
from io import BytesIO
import pdfplumber

# data analysis
import numpy as np
import pandas as pd
import re
import csv

import scraper

1.281070098205215


In [27]:
smic_urls_file = 'urls/smic_urls.csv'
smic_urls_reader = csv.reader(open( 'urls/smic_urls.csv', 'r', encoding='utf-8-sig'))
smic_urls = dict([(row[0] + " " + row[1], row[2]) for row in smic_urls_reader])

In [3]:
smic_urls.items()

dict_items([('2004 1', 'https://smic.shwebspace.com/uploads/qe/201101261833457_en.pdf'), ('2004 2', 'https://smic.shwebspace.com/uploads/qe/201101261843297_en.pdf'), ('2004 3', 'https://smic.shwebspace.com/uploads/qe/201101261848107_en.pdf'), ('2004 4', 'https://smic.shwebspace.com/uploads/qe/201101271517168_en.pdf'), ('2005 1', 'https://smic.shwebspace.com/uploads/qe/201101271532437_en.pdf'), ('2005 2', 'https://smic.shwebspace.com/uploads/qe/201101271535438_en.pdf'), ('2005 3', 'https://smic.shwebspace.com/uploads/qe/201101271540338_en.pdf'), ('2005 4', 'https://smic.shwebspace.com/uploads/qe/201101271545318_en.pdf'), ('2006 1', 'https://smic.shwebspace.com/uploads/qe/201101271554088_en.pdf'), ('2006 2', 'https://smic.shwebspace.com/uploads/qe/201101271556328_en.pdf'), ('2006 3', 'https://smic.shwebspace.com/uploads/qe/201101271608178_en.pdf'), ('2006 4', 'https://smic.shwebspace.com/uploads/qe/201101271636548_en.pdf'), ('2007 1', 'https://smic.shwebspace.com/uploads/qe/2011012716401

In [4]:
for quarter, url in smic_urls.items():
    print(quarter, url)
    if url not in ('NA', 'Link'):
        scraper.parse_smic(url)

2004 1 https://smic.shwebspace.com/uploads/qe/201101261833457_en.pdf
inv broken
segment broken
firstColName
DEBUG: successful
2004 2 https://smic.shwebspace.com/uploads/qe/201101261843297_en.pdf
inv broken
segment broken
firstColName
DEBUG: successful
2004 3 https://smic.shwebspace.com/uploads/qe/201101261848107_en.pdf
inv broken
segment broken
firstColName
DEBUG: successful
2004 4 https://smic.shwebspace.com/uploads/qe/201101271517168_en.pdf
inv broken
segment broken
firstColName
DEBUG: successful
2005 1 https://smic.shwebspace.com/uploads/qe/201101271532437_en.pdf
segment broken
firstColName
DEBUG: successful
2005 2 https://smic.shwebspace.com/uploads/qe/201101271535438_en.pdf
segment broken
firstColName
DEBUG: successful
2005 3 https://smic.shwebspace.com/uploads/qe/201101271540338_en.pdf
segment broken
firstColName
DEBUG: successful
2005 4 https://smic.shwebspace.com/uploads/qe/201101271545318_en.pdf
segment broken
firstColName
DEBUG: successful
2006 1 https://smic.shwebspace.com/u

In [36]:
url = "https://smic.shwebspace.com/uploads/qe/201101271745478_en.pdf"
rq = requests.get(url)
pdf = pdfplumber.open(BytesIO(rq.content))
smic_text = ""
for i in range(5):
    smic_text += pdf.pages[i].extract_text()

In [37]:
smic_text

' \n \n \nInvestor Contacts:            \nEn-Ling Feng  Anne Wong Chen  Edith Kwan \n+86-21-3861-0000 ext. 16275  +86-21-3861-0000 ext. 12804  +852-2116-2624 \nEnling_Feng@smics.com  Anne_CAYW@smics.com  Edith_Kwan@smics.com \n \n        \nNEWS RELEASE \n \nSMIC Reports 2009 Second Quarter Results \n \nAll currency figures stated in this report are in US Dollars unless stated otherwise. \nThe financial statement amounts in this report are determined in accordance with US GAAP. \n \nShanghai, China – July 28, 2009.  Semiconductor Manufacturing International Corporation (NYSE: \nSMI; SEHK: 981) (“SMIC” or the “Company”), one of the leading semiconductor foundries in the world, \ntoday announced its consolidated results of operations for the three months ended June 30, 2009.   \n \nSecond Quarter 2009 Highlights: \n \n(cid:1)  Revenue up by 82.5% to $267.4 million from $146.5 million in 1Q09 and down by 22.0% \ncompared to 2Q08.  Specifically, advanced logic sales from 0.13 micron and 90 

In [31]:
def clean_smic_seg(smic_text):
    digits = "\s+(\d+(?:\.\d+)?%)\s+(\d+(?:\.\d+)?%)\s+(\d+(?:\.\d+)?%)"
    quarters = "\s+(\d+Q\d+)\s+(\d+Q\d+)\s+(\d+Q\d+)"
    segCols = np.asarray(re.findall(f"(By (?:Service|Customer) Type){digits}", smic_text)[0])
    fabless = np.asarray(re.findall(f"(Fabless).*?{digits}", smic_text)[0])
    idm = np.asarray(re.findall(f"(Integrated device manufacturers).*?{digits}", smic_text)[0])
    systemComp = np.asarray(re.findall(f"(System companies and (?:O|o)thers){digits}", smic_text)[0])
    tsmcPlat = np.array([fabless, idm, systemComp])
    platDF = pd.DataFrame(tsmcPlat, columns=segCols)
    return platDF

In [32]:
clean_smic_seg(smic_text)

IndexError: list index out of range